<a href="https://www.kaggle.com/code/pedroalvess/student-performance-deep-dive?scriptVersionId=100439084" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/student-performance-data/student_data.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 60)
import numpy as np

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('../input/student-performance-data/student_data.csv')

# Exploratory Data Analysis

In [4]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [5]:
df['G_final'] = ((df['G1'] + df['G2'] + df['G3']) / 3).round(2)

In [6]:
fig = px.box(data_frame = df, y = ['G1', 'G2', 'G3', 'G_final'],
            template = 'plotly_dark')

fig.update_layout(
    title = {
        'text': "Box-plots das notas"},
    xaxis_title = 'Notas',
    yaxis_title = 'Valores',
    width = 1200,
    height = 500,
    font_family="Arial",
    font_color="White",
    title_font_family="Arial",
    title_font_color= "White")

fig.show()

In [7]:
df = df[(df['G2'] != 0) & (df['G3'] != 0)]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 0 to 394
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      357 non-null    object 
 1   sex         357 non-null    object 
 2   age         357 non-null    int64  
 3   address     357 non-null    object 
 4   famsize     357 non-null    object 
 5   Pstatus     357 non-null    object 
 6   Medu        357 non-null    int64  
 7   Fedu        357 non-null    int64  
 8   Mjob        357 non-null    object 
 9   Fjob        357 non-null    object 
 10  reason      357 non-null    object 
 11  guardian    357 non-null    object 
 12  traveltime  357 non-null    int64  
 13  studytime   357 non-null    int64  
 14  failures    357 non-null    int64  
 15  schoolsup   357 non-null    object 
 16  famsup      357 non-null    object 
 17  paid        357 non-null    object 
 18  activities  357 non-null    object 
 19  nursery     357 non-null    o

Perguntas:

1. Qual a média das notas G1, G2 e G3 dos alunos?
2. Estudantes com maior free time possuem em média notas mais altas?
3. O gênero do estudante impacta em seu desempenho?
4. Os estudantes possuem em média maior desempenho acadêmico sob a guarda paterna ou materna?
5. Como as faltas dos estudantes impactam na média das 3 notas?
6. Quais são os tipos de empregos predominantes entre os pais dos alunos?

**1. Qual a média das notas G1, G2 e G3 dos alunos?**

In [9]:
df['G1'].mean()

11.268907563025211

In [10]:
df['G2'].mean()

11.358543417366947

In [11]:
df['G3'].mean()

11.523809523809524

In [12]:
df['G_final'].mean()

11.38358543417367

In [13]:
print('As médias de notas dos alunos são {}, {} e {}, respectivamente.'.format(df['G1'].mean().round(2),
                                                                               df['G2'].mean().round(2),
                                                                               df['G3'].mean().round(2)))

As médias de notas dos alunos são 11.27, 11.36 e 11.52, respectivamente.


**2. Quais as quantidades de free time ideais para alunos?**

Para determinar isto, será feita uma análise de quais quantidades de free time há notas em média, maiores que a média das notas G1, G2 e G3.

In [14]:
df1 = df.groupby(by = 'freetime').agg({'G_final':'mean', 'G1':'mean', 'G2':'mean', 'G3':'mean'}).reset_index()
df1.sort_values(by = 'G_final')

,freetime,G_final,G1,G2,G3
0,1,10.863529,10.647059,10.941176,11.000000
2,3,11.061176,10.801471,11.088235,11.294118
3,4,11.207358,11.113208,11.198113,11.311321
4,5,11.912105,12.078947,11.763158,11.894737
1,2,12.238333,12.266667,12.116667,12.333333


In [15]:
fig = px.bar(data_frame = df1,
             x = 'freetime', y = 'G_final',
             text_auto = '.3s', template = 'plotly_dark')

fig.update_layout(
    title = {
        'text': "Média de notas por tempo livre"},
    xaxis_title = 'Tempo Livre',
    yaxis_title = 'Média de Notas',
    width = 1200,
    height = 500,
    font_family = "Arial",
    font_color = "White",
    font = dict(size = 18),
    title_font_family = "Arial",
    title_font_color = "White")

fig.update_traces(textposition = "outside", textfont_size = 16, cliponaxis = True)

fig.show()

Os estudantes que possuem 5 horas de free time, possuem notas acima da média.

**3. O sexo do estudante impacta em seu desempenho?**

In [16]:
df2 = df.groupby(by = 'sex').agg({'G1':'mean',
                            'G2':'mean',
                            'G3':'mean',
                            'G_final':'mean'}).reset_index().sort_values(by = ['G_final', 'G1', 'G2', 'G3'],
                                                                        ascending = False)
df2

,sex,G1,G2,G3,G_final
1,M,11.598837,11.709302,11.866279,11.724709
0,F,10.962162,11.032432,11.205405,11.066432


In [17]:
fig = make_subplots(rows = 2, cols = 2)

fig = make_subplots(rows = 2,
                    cols = 2,
                    subplot_titles = [
                        'G1',  # 1. título do subplot 
                        'G2',# 2. título do subplot
                        'G3',   # 3. título do subplot
                        'G médio', #  4. título do subplot
])

# Adiciona o primeiro grafico e especifica a linha e a coluna
fig.add_trace(go.Bar(x = df2['sex'], 
                     y = df2['G1'], textposition='auto',
                     name = 'G1'), row = 1, col = 1)

# adiciona o segundo gráfico
fig.add_trace(go.Bar(x = df2['sex'], 
                     y = df2['G2'], textposition='auto',
                     name = 'G2'), row = 1, col = 2)

# adiciona o terceiro gráfico
fig.add_trace(go.Bar(x = df2['sex'], 
                     y = df2['G3'], textposition='auto',
                     name = 'G3'), row = 2, col = 1)

# adiciona o quarto gráfico
fig.add_trace(go.Bar(x = df2['sex'], 
                     y = df2['G_final'], textposition='auto',
                     name = 'G Médio'), row = 2, col = 2)

# Update xaxis propriedades
fig.update_xaxes(title_text = "Sexos", row=1, col=1)
fig.update_xaxes(title_text = "Sexos", row=1, col=2)
fig.update_xaxes(title_text = "Sexos", row=2, col=1)
fig.update_xaxes(title_text = "Sexos", row=2, col=2)

# Update yaxis propriedades
fig.update_yaxes(title_text = "Nota G1", row=1, col=1)
fig.update_yaxes(title_text = "Nota G2", row=1, col=2)
fig.update_yaxes(title_text = "Nota G3", row=2, col=1)
fig.update_yaxes(title_text = "Nota G Médio", row=2, col=2)

fig.update_layout(title = 'Notas por Sexos', title_x = 0.5)

Os estudantes do sexo masculino possuem maiores notas, em média.

**4. Os estudantes possuem em média maior desempenho acadêmico sob a guarda paterna ou materna?**

In [18]:
df3 = df.groupby(by = 'guardian').agg({'G1':'mean',
                            'G2':'mean',
                            'G3':'mean',
                            'G_final':'mean'}).reset_index().sort_values(by = ['G_final', 'G1', 'G2', 'G3'],
                                                                        ascending = False)
df3

,guardian,G1,G2,G3,G_final
0,father,11.378049,11.573171,11.731707,11.560854
1,mother,11.250000,11.366935,11.540323,11.385484
2,other,11.111111,10.629630,10.740741,10.827778


Os estudantes sob a guarda do pai possuem as maiores notas, enquanto aqueles que estão sob a guarda de indivíduos que não são os seus pais ou mães, apresentam menor desempenho.

**5. Como a falta dos estudantes impactam na média das 3 notas?**

In [19]:
df4 = df.groupby(by = 'absences').agg({'G1':'mean',
                            'G2':'mean',
                            'G3':'mean',
                            'G_final':'mean'}).reset_index().sort_values(by = ['absences'])
df4

,absences,G1,G2,G3,G_final
0,0,12.038961,12.467532,12.610390,12.372597
1,1,12.333333,12.666667,13.000000,12.666667
2,2,11.630769,11.800000,11.907692,11.779385
3,3,11.500000,11.500000,11.500000,11.500000
4,4,10.830189,11.113208,11.283019,11.075094
5,5,11.200000,11.000000,11.600000,11.268000
6,6,10.838710,11.419355,11.774194,11.344194
7,7,11.428571,11.285714,11.857143,11.522857
8,8,10.863636,10.454545,10.590909,10.636364
9,9,13.333333,12.666667,13.333333,13.110000


In [20]:
fig = px.bar(data_frame = df4,
             x = 'absences', y = 'G_final',
             text_auto = '.3s', template = 'plotly_dark')

fig.update_layout(
    title = {
        'text': "Média de notas por faltas"},
    xaxis_title = 'Número de Faltas',
    yaxis_title = 'Média de Notas',
    width = 1200,
    height = 500,
    font_family = "Arial",
    font_color = "White",
    title_font_family = "Arial",
    title_font_color = "White")

fig.update_traces(textposition="outside", textfont_size = 18, cliponaxis=True)

fig.show()

**5.1 Quantas faltas os estudantes de maior desempenho possuem em média?**

In [21]:
df4_plot = df4[df4['G_final'] > df4['G_final'].mean()].sort_values(by = 'G_final',
                                                       ascending = False)
df4_plot

,absences,G1,G2,G3,G_final
24,24,18.000000,18.000000,18.000000,18.000000
21,21,17.000000,18.000000,18.000000,17.670000
9,9,13.333333,12.666667,13.333333,13.110000
23,23,13.000000,13.000000,13.000000,13.000000
1,1,12.333333,12.666667,13.000000,12.666667
0,0,12.038961,12.467532,12.610390,12.372597
13,13,11.666667,11.666667,12.666667,12.000000
11,11,12.333333,11.666667,11.666667,11.886667
20,20,12.750000,11.250000,11.500000,11.832500
2,2,11.630769,11.800000,11.907692,11.779385


In [22]:
df4_plot.sort_values(by = 'absences')

,absences,G1,G2,G3,G_final
0,0,12.038961,12.467532,12.610390,12.372597
1,1,12.333333,12.666667,13.000000,12.666667
2,2,11.630769,11.800000,11.907692,11.779385
3,3,11.500000,11.500000,11.500000,11.500000
4,4,10.830189,11.113208,11.283019,11.075094
5,5,11.200000,11.000000,11.600000,11.268000
6,6,10.838710,11.419355,11.774194,11.344194
7,7,11.428571,11.285714,11.857143,11.522857
9,9,13.333333,12.666667,13.333333,13.110000
10,10,11.176471,10.764706,11.235294,11.058235


Há um range de 0 a 13 faltas onde os estudantes possuem maiores notas.

In [23]:
fig = px.bar(data_frame = df4_plot,
             x = 'absences', y = 'G_final',
             text_auto = '.3s', template = 'plotly_dark')

fig.update_layout(
    title = {
        'text': "Média de notas dos alunos acima da média por faltas"},
    xaxis_title = 'Número de Faltas',
    yaxis_title = 'Média de Notas',
    width = 1200,
    height = 500,
    font_family = "Arial",
    font_color = "White",
    font = dict(size = 18),
    title_font_family = "Arial",
    title_font_color = "White")

fig.update_traces(textposition="outside", textfont_size = 16, cliponaxis=True)

fig.show()

**6. Quais são os tipos de empregos predominantes entre os pais dos alunos?**

In [24]:
# criando coluna auxiliar para realizar contagens
df['count'] = 1

In [25]:
df5 = df.groupby(by = ['Fjob']).agg({'count':'count'}).reset_index().sort_values(by = 'count',
                                                                          ascending = False)
df5['percent'] = ((df5['count'] / df5['count'].sum())*100).round(2)

In [26]:
df5['percent'] = df5['percent'].apply(lambda x: '{0:1.2f}%'.format(x))

In [27]:
df5

,Fjob,count,percent
2,other,196,54.90%
3,services,100,28.01%
4,teacher,26,7.28%
1,health,18,5.04%
0,at_home,17,4.76%


In [28]:
df5 = df5.reindex([3, 4, 1, 0, 2])

In [29]:
df5['Fjob'] = df5['Fjob'].apply(lambda x: 'Services' if x == 'services' else
                            'Teacher' if x == 'teacher' else
                            'Health' if x == 'health' else
                            'At Home' if x == 'at_home' else
                            'Others')

In [30]:
df5

,Fjob,count,percent
3,Services,100,28.01%
4,Teacher,26,7.28%
1,Health,18,5.04%
0,At Home,17,4.76%
2,Others,196,54.90%


In [31]:
fig = px.bar(data_frame = df5,
             x = 'Fjob', y = 'count',
             text_auto = True, template = 'plotly_dark')

fig.update_layout(
    title = {
        'text': 'Tipos de trabalho dos Pais'},
    xaxis_title = 'Tipos de trabalho',
    yaxis_title = 'Quantidade de Pais',
    width = 1200,
    height = 500,
    font_family = "Arial",
    font = dict(size = 18),
    font_color = "White",
    title_font_family = "Arial",
    title_font_color = "White")

fig.update_traces(textposition="outside", textfont_size = 16, cliponaxis=True)

fig.show()

Os tipos de trabalho predominantes dos pais das crianças estão na categoria services e professor, enquanto que as mães também se enquadram em services, mas realizam trabalhos domésticos como segunda maior colocação. Ambos os sexos possuem trabalhos diversos.

In [32]:
df6 = df.groupby(by = ['Mjob']).agg({'count':'count'}).reset_index().sort_values(by = 'count',
                                                                          ascending = False)

In [33]:
df6

,Mjob,count
2,other,127
3,services,94
4,teacher,54
0,at_home,50
1,health,32


In [34]:
df6 = df6.reindex([3, 4, 0, 1, 2])

In [35]:
df6['Mjob'] = df6['Mjob'].apply(lambda x: 'Services' if x == 'services' else
                            'Teacher' if x == 'teacher' else
                            'Health' if x == 'health' else
                            'At Home' if x == 'at_home' else
                            'Others')

In [36]:
fig = px.bar(data_frame = df6,
             x = 'Mjob', y = 'count',
             text_auto = True, template = 'plotly_dark')

fig.update_layout(
    title = {
        'text': 'Tipos de trabalho das Mães'},
    xaxis_title = 'Tipos de trabalho',
    yaxis_title = 'Quantidade de Mães',
    width = 1200,
    height = 500,
    font_family = "Arial",
    font_color = "White",
    font = dict(size = 18),
    title_font_family = "Arial",
    title_font_color = "White")

fig.update_traces(textposition="outside", textfont_size = 16, cliponaxis=True)

fig.show()

In [37]:
fig1 = px.histogram(data_frame = df,
             x = 'G1', nbins = 25, template = 'plotly_dark')

fig2 = px.histogram(data_frame = df,
             x = 'G2', nbins = 25, template = 'plotly_dark')

fig3 = px.histogram(data_frame = df,
             x = 'G3', nbins = 25, template = 'plotly_dark')

fig1.update_layout(
    title = {
        'text': "Distribuição das notas (G1)"},
    xaxis_title = 'Notas',
    yaxis_title = 'Frequência',
    width = 1000,
    height = 500,
    font_family="Arial",
    font_color="White",
    font = dict(size = 18),
    title_font_family="Arial",
    title_font_color= "White",
    legend_title_font_color = "green")

fig1.update_traces(textposition="outside", textfont_size = 16, cliponaxis=True)

fig2.update_layout(
    title = {
        'text': "Distribuição das notas (G2)"},
    xaxis_title = 'Notas',
    yaxis_title = 'Frequência',
    width = 1000,
    height = 500,
    font_family="Arial",
    font_color="White",
    font = dict(size = 18),
    title_font_family="Arial",
    title_font_color= "White",
    legend_title_font_color = "green")

fig2.update_traces(textposition="outside", textfont_size = 16, cliponaxis=True)

fig3.update_layout(
    title = {
        'text': "Distribuição das notas (G3)"},
    xaxis_title = 'Notas',
    yaxis_title = 'Frequência',
    width = 1000,
    height = 500,
    font_family="Arial",
    font_color="White",
    font = dict(size = 18),
    title_font_family="Arial",
    title_font_color= "White",
    legend_title_font_color = "green")

fig3.update_traces(textposition="outside", textfont_size = 16, cliponaxis=True)

fig1.show()
fig2.show()
fig3.show()

As notas dos alunos seguem uma distribuição normal.

# Label Enconder

In [38]:
lb = LabelEncoder()

In [39]:
objList = df.select_dtypes(include = "object").columns

In [40]:
for obj in objList:
    df[obj] = lb.fit_transform(df[obj].astype(str))

In [41]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G_final,count
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6,5.67,1
1,0,0,17,1,0,1,1,1,0,2,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6,5.33,1
2,0,0,15,1,1,1,1,1,0,2,2,1,1,2,3,1,0,1,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10,8.33,1
3,0,0,15,1,0,1,4,2,1,3,1,1,1,3,0,0,1,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15,14.67,1
4,0,0,16,1,0,1,3,3,2,2,1,0,1,2,0,0,1,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10,8.67,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,20,1,1,0,2,2,3,3,0,2,1,2,2,0,1,1,0,1,1,0,0,5,5,4,4,5,4,11,9,9,9,9.00,1
391,1,1,17,1,1,1,3,1,3,3,0,1,2,1,0,0,0,0,0,0,1,1,0,2,4,5,3,4,2,3,14,16,16,15.33,1
392,1,1,21,0,0,1,1,1,2,2,0,2,1,1,3,0,0,0,0,0,1,0,0,5,5,3,3,3,3,3,10,8,7,8.33,1
393,1,1,18,0,1,1,3,2,3,2,0,1,3,1,0,0,0,0,0,0,1,1,0,4,4,1,3,4,5,0,11,12,10,11.00,1


# 

# Linear Regression

In [42]:
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'G_final', 'count'],
      dtype='object')

In [43]:
cols = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences']

In [44]:
X = df[cols]
y = df[['G_final']]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [46]:
model = LinearRegression()

In [47]:
model.fit(X_train, y_train)

LinearRegression()

In [48]:
model.score(X_train, y_train)

0.2756843154798406

In [49]:
model.score(X_test, y_test)

0.21553127377936754

In [50]:
y_pred = model.predict(X_test)

# Decision Tree

In [51]:
rg = DecisionTreeRegressor(random_state = 8254)

In [52]:
rg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=8254)

In [53]:
rg.score(X_train, y_train)

1.0

In [54]:
rg.score(X_test, y_test)

-0.7629538527214632